In [1]:
import pandas as pd
import rdkit
from chembl_webresource_client.new_client import new_client

In [44]:
# Создаем API, фильтруя результаты по id таргетной молекулы и типу активности

activity_api= new_client.activity.filter(target_chembl_id='CHEMBL230', type='IC50').only('molecule_chembl_id','canonical_smiles', 'standard_relation', 'standard_value', 'standard_units')
molecule_data = list(activity_api)
            
# Создание DataFrame

df = pd.DataFrame(molecule_data)

# Проверка SMILES

def smiles_check(smiles_str):
    if rdkit.Chem.MolFromSmiles(smiles_str) is None:
        return False
    else:
        return True

df = df.dropna(subset=['canonical_smiles', 'standard_value']) # Удаляем строчки со значением Nan в столбце 'canonical_smiles'

df = df[df['canonical_smiles'].apply(smiles_check) == True]

# Перевод строчки 'standard_value' в численный формат

df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')

# Отфильтровываем значения в nM

df = df[df['standard_units'] == 'nM']

# Удаляем дубликаты и объекты с пустыми/нулевыми значениями столбца 'standard_value'

res = df[(df['standard_value'] != 0) & (df['standard_value'] != '')]

res = pd.DataFrame.drop_duplicates(res, subset='molecule_chembl_id', keep='first')

# Фильтруем объекты по значениям 'standard_relation'

res = res[res['standard_relation'] == '=']

# Запись результата в .csv файл

with open('Parsing_res.csv', 'w', newline='', encoding='utf-8'):
    res.to_csv('Parsing_res.csv', columns = ['canonical_smiles', 'standard_value'], index = False)